In [1]:
import math
from datetime import datetime

import bw2data as bd
import numpy as np
import pandas as pd
import pytest
from bw2data.tests import bw2test
from bw_temporalis import TemporalDistribution

from bw_timex import TimexLCA


bd.projects.set_current("__wastes_db__")
biosphere = bd.Database("biosphere")
biosphere.write(
    {
        ("biosphere", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    }
)

foreground = bd.Database("foreground")
foreground.register()
background_2020 = bd.Database("background_2020")
background_2020.register()

node_co2 = biosphere.get("CO2")

fu = foreground.new_node("fu", name="fu", unit="unit")
fu["reference product"] = "fu"
fu.save()

used_car = foreground.new_node("used_car", name="used_car", unit="unit")
used_car["reference product"] = "used_car"
used_car.save()

car_treatment_2020 = background_2020.new_node(
    "car_treatment", name="car_treatment", unit="car_treatment"
)
car_treatment_2020["reference product"] = "car_treatment"
car_treatment_2020.save()

fu.new_edge(input=fu, amount=1, type="production").save()
fu.new_edge(input=used_car, amount=-1, type="technosphere").save()

used_car.new_edge(input=used_car, amount=-1, type="production").save()
used_car.new_edge(input=car_treatment_2020, amount=-3, type="technosphere").save()

car_treatment_2020.new_edge(
    input=car_treatment_2020, amount=-1, type="production"
).save()
car_treatment_2020.new_edge(input=node_co2, amount=1, type="biosphere").save()

bd.Method(("GWP", "example")).write(
    [
        (("biosphere", "CO2"), 1),
    ]
)

100%|██████████| 1/1 [00:00<00:00, 7281.78it/s]

Vacuuming database 


In [2]:
method = ("GWP", "example")
database_date_dict = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "foreground": "dynamic",
}
fu = ("foreground", "fu")
tlca = TimexLCA({fu: 1}, method, database_date_dict)
tlca.build_timeline()
tlca.lci()
tlca.static_lcia()

assert math.isclose(tlca.static_lca.score, tlca.static_score, rel_tol=1e-9)

Starting graph traversal
Calculation count: 2


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timeline_builder.py:527: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [5]:
tlca.timeline

,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2024,5,2024-01-01,2,fu,2024,-1,2024-01-01,-1,-1,1.0,None
1,2024,6,2024-01-01,3,used_car,2024,5,2024-01-01,2,fu,-1.0,None
2,2024,7,2024-01-01,4,car_treatment,2024,6,2024-01-01,3,used_car,-3.0,{'background_2020': 1}


In [3]:
tlca.static_lca.score

3.0

In [4]:
tlca.static_score

3.0000000000000004